In [13]:
# Date of creation: 12-12-24
# Author: David Yang

This will be dedicated to generating the following pieces of data for the team at Google Deep Mind.
The goal is to generate bed file(s) which contain information about the methylation patterns of my MSC scamples.
-  A bed file of "high-confidence" consensus unmethylated regions --> generated using AlleleStacker algorithm for candidate region generation with increased min_sample count, no filtering
- A bed file of high confidence methylation regions (generated the same way)

* Ill all samples (n=18)

### **Step 1: CpG Methylation Probability Pileup QC**

CpG methylation probability pileups generated by [pb-CpG-tools](https://github.com/PacificBiosciences/pb-CpG-tools) (v2.3.2) can produce incorrect results in two key scenarios where there is a heterozygous variant:

1. **Variant Sites Appearing as Unmethylated CpGs**: When genetic variants disrupt CpG sites, these positions can be misinterpreted as CpGs with 0%/low methylation (>10%) probability. This creates "phantom" unmethylated CpG calls at variant locations. 
2. **Denovo CpG Creation**: Variants can create new CpG sites on one haplotype that don't exist on the other. This leads to incorrect methylation calls when the non-CpG haplotype is interpreted as having unmethylated CpGs.

#### Workflow

**Input Files Required (specified in config.yaml):**
- VCF file containing genetic variants
- Reference FASTA file
- CpG methylation BED files (combined, hap1, hap2)

**Analysis Steps:**
1. Identify variant positions affecting CpG sites
2. Detect destroyed reference CpGs and created de novo CpGs per haplotype
3. Filter methylation calls based on variant effects
4. Generate QC metrics and visualizations

**Output Files:**
- Filtered BED files with problematic sites removed
- Excluded sites BED files
- QC report with filtering statistics
- Distribution plots for passing and excluded sites


### Filter out phantom CpG sites:

In [10]:
# Define the base directory
base_dir = "/gs/gsfs0/users/greally-lab/David/AlleleStacker_tests/January_2025/GDM/AlleleStacker"

In [11]:
# Construct paths using the base directory
bash_dir = f"{base_dir}/bash/pileup_QC"
sample_list = f"{base_dir}/bash/pileup_QC/sample_list.txt"
config_file = f"{base_dir}/bash/pileup_QC/config.yaml"
output_dir = f"{base_dir}/outputs/pileup_QC"
python_dir=f"{base_dir}/python/pileup_QC"

# Run the filtering script
!sbatch $bash_dir/submit_pileup_qc.sh $sample_list $config_file $output_dir $python_dir

Submitted batch job 8957174


### Visualize the results of the filtered CpG sites.

In [ ]:
# Update paths for summary plots.
import os
input_dir = f"{base_dir}/outputs/pileup_QC"
output_dir = f"{base_dir}/outputs/pileup_QC/summary_plots"
python_script = f"{base_dir}/python/pileup_QC/summarize_qc.py"

# Run the script
import subprocess
subprocess.run(["python", python_script, "--input-dir", input_dir, "--output-dir", output_dir])

#### Note
A site is considered "preserved" if it meets these criteria:
- Has high methylation probability (>90%)
- Has good coverage (≥10 reads)
- Was initially marked for exclusion but isn't at a variant position


### **Step 2: Generate segmentation data**


In [4]:
# Define the base directory
base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/January_2025/GDM/AlleleStacker"

In [14]:
import os
# Run the segmentation script on QCed CpG data
bash_dir = f"{base_dir}/bash/segmentation"
input_dir = f"{base_dir}/outputs/pileup_QC"
output_dir = f"{base_dir}/outputs/segmentation"

!sbatch $bash_dir/a_segment_samples.sh $input_dir $output_dir


Submitted batch job 8957195


In [15]:
# Parse the segmentation results and extract regions by segmentation label for downstream use
bash_dir = f"{base_dir}/bash/segmentation"
input_dir = f"{base_dir}/outputs/segmentation"
output_dir = f"{base_dir}/outputs/segmentation/regions_by_label"

!sbatch $bash_dir/b_extract-regions.sh $input_dir $output_dir 




Submitted batch job 8957214


### Generate plots of the segmented regions

In [ ]:
import os
# Summarize the segmentation results for each sample.
# Summary plots of the region counts and region sizes

bash_dir = f"{base_dir}/bash/segmentation"
input_dir = f"{base_dir}/outputs/segmentation"
output_dir = f"{base_dir}/outputs/segmentation"


!sbatch $bash_dir/plot_segmentation_sample.sh $input_dir $output_dir
!sbatch $bash_dir/plot_segmentation_group.sh $input_dir $output_dir


### **Step 3: Generate candidate regions of interest**
* This will be used to generate the bed files of high confidence regions.
Parameters:

For high confidence unmethylated regions: 
- min_samples = 10

For high confidence methylated regions: 
- min_samples = 10




In [12]:
# Define the base directory
base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/January_2025/GDM/AlleleStacker"

In [16]:
# Generate the consensus regions for unmethylated loci
bash_dir = f"{base_dir}/bash/candidate_regions"
input_dir = f"{base_dir}/outputs/segmentation/regions_by_label/regions"
output_dir = f"{base_dir}/outputs/candidate_regions/unmethylated_regions"

!sbatch $bash_dir/a_consensus_regions.sh $input_dir $output_dir

Submitted batch job 8957215


Unmethylated consensus stats:

H1 Statistics:
Average number of regions per sample: 75141.72 ± 10269.78
Average region size across samples: 746.60 ± 93.57
Consensus regions: 43210 regions
Consensus region size: 1682.99 ± 1252.05

H2 Statistics:
Average number of regions per sample: 75055.50 ± 10307.12
Average region size across samples: 745.04 ± 92.43
Consensus regions: 43211 regions
Consensus region size: 1678.62 ± 1234.70


In [17]:
# Generate the consensus regions for methylated loci
bash_dir = f"{base_dir}/bash/candidate_regions"
input_dir = f"{base_dir}/outputs/segmentation/regions_by_label/regions"
output_dir = f"{base_dir}/outputs/candidate_regions/methylated_regions"

!sbatch $bash_dir/a_consensus_regions_meth.sh $input_dir $output_dir

Submitted batch job 8957217



Methylated Consensus Info:

H1 Statistics:
Average number of regions per sample: 573740.11 ± 35880.79
Average region size across samples: 1888.95 ± 57.93
Consensus regions: 441682 regions
Consensus region size: 3441.46 ± 3319.46

H2 Statistics:
Average number of regions per sample: 573829.22 ± 35680.26
Average region size across samples: 1887.60 ± 57.69
Consensus regions: 441438 regions
Consensus region size: 3439.89 ± 3317.24


### **Step 4: Generate IGV viewable files**
For visualizing our candidate regions and variants, we will need to aggregate, format, and colorize the 2 datasets generated:
- Individual sample data
- Candidate regions

Files generated:

- allele_stacks :  IGV colorized bed files of each sample's segmentation regions organized by haplotype.
- candidate_regions : IGV colorized bed files of the candidate regions

In [6]:
# Define the base directory
base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/January_2025/GDM/AlleleStacker"

In [19]:
# Generate allele stacks of the segmentation by aggregating the segmentation data for each sample
# Set up paths
bash_dir = f"{base_dir}/bash/igv_viewing"
INPUT_DIR=f"{base_dir}/outputs/segmentation/regions_by_label/regions"
OUTPUT_DIR=f"{base_dir}/outputs/igv_beds/cohort_stack"
SAMPLE_LIST=f"{base_dir}/bash/pileup_QC/sample_list.txt"

!sbatch $bash_dir/a_IGV_all-samples.sh $INPUT_DIR $OUTPUT_DIR $SAMPLE_LIST


Submitted batch job 8957220


In [18]:
# Generate IGV beds for the unmethylated regions
# # Set up paths
bash_dir = f"{base_dir}/bash/igv_viewing"
INPUT_DIR=f"{base_dir}/outputs/candidate_regions/unmethylated_regions"
OUTPUT_DIR=f"{base_dir}/outputs/igv_beds/unmethylated_regions"


!sbatch $bash_dir/b_consensus-IGV.sh $INPUT_DIR $OUTPUT_DIR


Submitted batch job 8957219


In [21]:
# Generate IGV beds for the methylated regions
# # Set up paths
bash_dir = f"{base_dir}/bash/igv_viewing"
INPUT_DIR=f"{base_dir}/outputs/candidate_regions/methylated_regions"
OUTPUT_DIR=f"{base_dir}/outputs/igv_beds/methylated_regions"


!sbatch $bash_dir/b_consensus-IGV.sh $INPUT_DIR $OUTPUT_DIR


Submitted batch job 8957341


In [ ]:
# Create BED files for IGV viewing each sample's H1 and H2 segments
# Define paths
bash_dir = f"{base_dir}/bash/igv_viewing"
input_dir=f"{base_dir}/outputs/segmentation/regions_by_label/regions"
output_dir=f"{base_dir}/outputs/igv_beds/sample_segmentation"

!sbatch $bash_dir/c_IGV-segmentation.sh $input_dir $output_dir



### **Step 5: Prepare VCFs for variant mapping**
This requires merging each sample's VCF for each variant type into a cohort wide VCF.
* Adjust file paths within each bash script

Each VCF was QC'ed with the following parameters for this run :


Merging:
bcftools merge
    --force-samples \
    --merge both \
    --file-list "$FILEPATHS" \
    --output-type z \

Small variants:
bcftools filter -i 'QUAL>=20 && DP>=10' 
bcftools annotate --set-id '%CHROM:%POS:%REF:%ALT'

SVs:
bcftools filter -i 'FILTER="PASS" && DP>=10'
bcftools annotate --set-id '%CHROM:%POS:%REF:%ALT:%SVANN'

CNVs:
bcftools filter -i 'QUAL>=20 && FILTER="PASS"
bcftools annotate --set-id '%CHROM:%POS:%REF:%ALT:%SVLEN' 

Tandem Repeats:
bcftools filter -i 'QUAL>=20 && FILTER="PASS"'
bcftools annotate --set-id '%CHROM:%POS:%REF:%ALT'

In [42]:
# Define the base directory
base_dir = "/gs/gsfs0/users/greally-lab/David/AlleleStacker_tests/January_2025/AlleleStacker"

In [43]:
# merge each VCF type into a single cohort vcf using bcftools merge
# small variants

# Set up environment
import os
bash_dir = f"{base_dir}/bash/variant_map"
FILEPATHS = f"{base_dir}/bash/variant_map/file_paths/filepaths_small.txt"
OUT_DIR = f"{base_dir}/outputs/variant_mapping/merged_variants/small_variants"

!sbatch $bash_dir/a_merge_small_vars.sh $FILEPATHS $OUT_DIR

Submitted batch job 8954328


In [44]:
# merge structural variants

bash_dir = f"{base_dir}/bash/variant_map"
FILEPATHS = f"{base_dir}/bash/variant_map/file_paths/filepaths_SVs.txt"
OUT_DIR = f"{base_dir}/outputs/variant_mapping/merged_variants/structural_variants"

!sbatch $bash_dir/b_merge_sv_vars.sh $FILEPATHS $OUT_DIR

Submitted batch job 8954329


In [45]:
# merge copy number variants

bash_dir = f"{base_dir}/bash/variant_map"
FILEPATHS = f"{base_dir}/bash/variant_map/file_paths/filepaths_CNVs.txt"
OUT_DIR = f"{base_dir}/outputs/variant_mapping/merged_variants/copy_number_variants"

!sbatch $bash_dir/c_merge_CNVs_vars.sh $FILEPATHS $OUT_DIR

Submitted batch job 8954330


In [ ]:
# merge tandem repeats

bash_dir = f"{base_dir}/bash/variant_map"
FILEPATHS = f"{base_dir}/bash/variant_map/file_paths/filepaths_repeats.txt"
OUT_DIR = f"{base_dir}/outputs/variant_mapping/merged_variants/tandem-repeat_variants"

!sbatch $bash_dir/d_merge_repeats_vars.sh  $FILEPATHS $OUT_DIR



Submitted batch job 8954331


### Step 6: Map variants to regions

#### Haplotype-Specific Variant Mapping Analysis

Overview
Maps genetic variants to haplotype-specific methylated/unmethylated regions. Processes phased variants (SNPs, SVs) and unphased variants (CNVs, tandem repeats) from VCF files against methylation BED files.

Input Requirements
- BED file with methylation regions containing:
  - Chromosome, start, end positions
  - `methylated_samples` and `unmethylated_samples` columns
- Indexed VCF files (.vcf.gz + .tbi) for variants:
  - Small variants (SNPs/indels)
  - Copy number variants
  - Structural variants  
  - Tandem repeats

Output Format
Tab-separated file with columns:
```
chrom         Chromosome
start         Region start position
end           Region end position
variant_id    Unique variant identifier 
type          Variant type (small/cnv/sv/tr)
ref           Reference allele
alt           Alternative allele
num_meth      Number of samples with variant + methylated
num_unmeth    Number of samples with variant + unmethylated
meth_samples  Methylated samples with variant: sample:genotype
unmeth_samples Unmethylated samples with variant: sample:genotype
```
Haplotype Assignment
- Phased variants (small, sv): Uses `|`-separated genotypes, checking H1 (idx 0) or H2 (idx 1)
- Unphased variants (cnv, tr): Reports any non-reference genotype



### Run a test for 1,000 regions

In [ ]:

# Set up environment
import os
import sys

# Set up paths
base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker"
bash_dir ="/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/1-14-25/AlleleStacker/bash/variant_map"
BED_DIR=f"{base_dir}/outputs/candidate_regions/QC/filtered/test_1k_regions"
VARIANT_DIR=f"{base_dir}/outputs/variant_mapping/merged_variants"
PYTHON_DIR="/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/1-14-25/AlleleStacker/python/variant_map"
OUTPUT_DIR="/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/1-14-25/AlleleStacker/outputs/variant_mapping/1k_regions"  

# SUBMIT THE JOB
!sbatch $bash_dir/test_map.sh $BED_DIR $VARIANT_DIR $PYTHON_DIR $OUTPUT_DIR

### Run for all regions

In [ ]:

# Set up environment
import os
import sys

# Set up paths
base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker"
bash_dir ="/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/1-14-25/AlleleStacker/bash/variant_map"
BED_DIR=f"{base_dir}/outputs/candidate_regions/QC/filtered"
VARIANT_DIR=f"{base_dir}/outputs/variant_mapping/merged_variants"
PYTHON_DIR="/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/1-14-25/AlleleStacker/python/variant_map"
OUTPUT_DIR="/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/1-14-25/AlleleStacker/outputs/variant_mapping/all_regions"  

# SUBMIT THE JOB
!sbatch $bash_dir/test_map.sh $BED_DIR $VARIANT_DIR $PYTHON_DIR $OUTPUT_DIR

### DeepMind Figs

Date: 1-24-25



In [2]:
# Define the base directory
base_dir = "/gs/gsfs0/users/greally-lab/David/AlleleStacker_tests/January_2025/AlleleStacker"

In [6]:
# Generate visualization of the distribution of unmethylated and methylated regions for each CHROMOSOME
# Adjust bin size in python script:
bash_dir = f"{base_dir}/bash/segmentation"
input_dir = f"{base_dir}/outputs/segmentation/regions_by_label/regions"
# Adjust based on bin size
# output_dir = f"{base_dir}/outputs/segmentation/distribution_plots/mean_counts/bin_10Mb"
# output_dir = f"{base_dir}/outputs/segmentation/distribution_plots/mean_counts/bin_1Mb"
output_dir = f"{base_dir}/outputs/segmentation/distribution_plots/mean_counts/bin_100kb"


!sbatch $bash_dir/plot_region_distribution_counts.sh $input_dir $output_dir

Submitted batch job 8956848


In [3]:
# Generate visualization of the distribution of unmethylated and methylated regions for each CHROMOSOME
# Adjust bin size in python script:
bash_dir = f"{base_dir}/bash/segmentation"
input_dir = f"{base_dir}/outputs/segmentation/regions_by_label/regions"
# Adjust based on bin size
output_dir = f"{base_dir}/outputs/segmentation/distribution_plots/size/bin_10Mb"
# output_dir = f"{base_dir}/outputs/segmentation/distribution_plots/size/bin_1Mb"
# output_dir = f"{base_dir}/outputs/segmentation/distribution_plots/size/bin_100kb"


!sbatch $bash_dir/plot_region_distribution_size.sh $input_dir $output_dir

Submitted batch job 8956885


### OLD GRAPHICS BELOW

### Mermaid format diagram of the variant mapper logic 

In [ ]:

import graphviz

# Create directed graph
dot = graphviz.Digraph('Methylation and Variant States', 
                       comment='Sample State Diagram',
                       graph_attr={'rankdir': 'TD'})

# Main states
dot.node('A', 'States for Sample', shape='box')
dot.node('B', 'Methylation Status (M)', shape='doubleoctagon')
dot.node('C', 'Variant Status (V)', shape='doubleoctagon')

# Methylation states
dot.node('B1', 'M+: Methylated')
dot.node('B2', 'M-: Unmethylated')
dot.node('B3', 'X(M): Missing Methylation Data')

# Variant states
dot.node('C1', 'V+: Has Variant')
dot.node('C2', 'V-: No Variant')
dot.node('C3', 'X(V): Missing Variant Data')

# Connect main nodes
dot.edge('A', 'B')
dot.edge('A', 'C')
dot.edge('B', 'B1')
dot.edge('B', 'B2')
dot.edge('B', 'B3')
dot.edge('C', 'C1')
dot.edge('C', 'C2')
dot.edge('C', 'C3')

# Create subgraph for combined states
with dot.subgraph(name='cluster_0') as c:
    c.attr(label='Combined States', style='dashed')
    for state in [
        ('D1', 'M+V+'), ('D2', 'M+V-'), ('D3', 'M+X(V)'),
        ('D4', 'M-V+'), ('D5', 'M-V-'), ('D6', 'M-X(V)'),
        ('D7', 'X(M)V+'), ('D8', 'X(M)X(V)')
    ]:
        c.node(state[0], state[1], shape='circle')

# Add descriptions
descriptions = {
    'G1': 'M+V+: Methylated with variant',
    'G2': 'M-V+: Unmethylated with variant',
    'G3': 'M+V-: Methylated without variant',
    'G4': 'M-V-: Unmethylated without variant',
    'G5': 'X(M-)V+: No methylation data, has variant',
    'G6': 'X(V-)M+: No variant data, methylated',
    'G7': 'X(V-)M-: No variant data, unmethylated',
    'G8': 'X(M-)X(V-): No data for either'
}

# Add description nodes and connect
for i, (key, desc) in enumerate(descriptions.items(), 1):
    dot.node(key, desc, shape='box')
    dot.edge(f'D{i}', key)

# Display graph
dot

In [21]:
# First diagram: State relationships
def create_state_diagram():
    dot = graphviz.Digraph('Methylation and Variant States',
                          comment='Sample State Diagram',
                          graph_attr={'rankdir': 'TB', 'nodesep': '0.5', 'ranksep': '0.7'})

    # Main states in top rank
    with dot.subgraph(name='cluster_top') as top:
        top.attr(rank='source')
        top.node('A', 'States for Sample', shape='box')

    # Methylation and Variant status in second rank
    with dot.subgraph() as second:
        second.attr(rank='same')
        second.node('B', 'Methylation Status (M)', shape='doubleoctagon')
        second.node('C', 'Variant Status (V)', shape='doubleoctagon')

    # Connect main to second rank
    dot.edge('A', 'B')
    dot.edge('A', 'C')

    # Individual states in third rank
    with dot.subgraph() as third:
        third.attr(rank='same')
        # Methylation states
        third.node('B1', 'M+: Methylated')
        third.node('B2', 'M-: Unmethylated')
        third.node('B3', 'X(M): Missing Methylation Data')
        # Variant states
        third.node('C1', 'V+: Has Variant')
        third.node('C2', 'V-: No Variant')
        third.node('C3', 'X(V): Missing Variant Data')

    # Connect states
    dot.edge('B', 'B1')
    dot.edge('B', 'B2')
    dot.edge('B', 'B3')
    dot.edge('C', 'C1')
    dot.edge('C', 'C2')
    dot.edge('C', 'C3')

    # Combined states at bottom
    with dot.subgraph(name='cluster_combined') as c:
        c.attr(label='Combined States', style='dashed')
        c.attr(rank='sink')
        states = [
            ('D1', 'M+V+'), ('D2', 'M+V-'), ('D3', 'M+X(V)'),
            ('D4', 'M-V+'), ('D5', 'M-V-'), ('D6', 'M-X(V)'),
            ('D7', 'X(M)V+'), ('D8', 'X(M)X(V)')
        ]
        for state in states:
            c.node(state[0], state[1], shape='circle')

    return dot

# Second diagram: Scoring Logic
def create_scoring_diagram():
    dot = graphviz.Digraph('Methylation Scoring Logic',
                          comment='Scoring System Diagram',
                          graph_attr={'rankdir': 'TB', 'nodesep': '0.8', 'ranksep': '1.0'})

    # State descriptions with scoring implications
    descriptions = {
        'G1': '''M+V+ State
• Methylated with variant
• Primary contributor to methylation score
• Weight: 40% of ratio''',
        
        'G2': '''M-V+ State
• Unmethylated with variant
• Primary contributor to unmethylation score
• Weight: 60% of inverse ratio''',
        
        'G3': '''Baseline States (M+V-, M-V-)
• Used for ratio denominators
• Define expected methylation patterns''',
        
        'G4': '''Missing Data States
• X(M)V+: Reduces confidence
• X(V)M+/-, X(M)X(V): Not scored'''
    }

    # Add state descriptions
    with dot.subgraph(name='cluster_states') as s:
        s.attr(label='State Contributions', style='dashed')
        for key, desc in descriptions.items():
            s.node(key, desc, shape='box')

    # Add scoring components
    with dot.subgraph(name='cluster_scoring') as s:
        s.attr(label='Scoring System', style='dashed')
        
        s.node('S1', '''Methylation Score Components
------------------------
1. Methylation Ratio (R_m)
   M+V+ samples / Total M+ samples
   Weight: 0.4

2. Unmethylation Protection (R_u)
   1 - (M-V+ samples / Total M- samples)
   Weight: 0.6

3. Statistical Significance
   -0.1 * log10(p-value)''', shape='note')
        
        s.node('S2', '''Confidence Weighting
------------------------
• High (>80%): 1.0
• Medium (>60%): 0.7
• Low (>40%): 0.4
• Very Low: 0.1''', shape='note')
        
        s.node('S3', '''Final Score Calculation
------------------------
Score = confidence_weight * 
(0.4*R_m + 0.6*(1-R_u)) - 
0.1*log10(p-value)''', shape='note')

    # Connect components
    dot.edge('G1', 'S1')
    dot.edge('G2', 'S1')
    dot.edge('S1', 'S3')
    dot.edge('S2', 'S3')

    return dot

# Create both diagrams
state_diagram = create_state_diagram()
scoring_diagram = create_scoring_diagram()

In [ ]:
# To show state diagram:
state_diagram

In [ ]:
# To show scoring diagram:
scoring_diagram